### IMPORTS

In [8]:
import sys, os
sys.path.append(os.path.join(sys.path[0],'Pythons'))

In [9]:
from Carga_Descarga import *
from CRM_Central import *

### QUERIES

In [10]:
q = '''WITH barter_table AS (
    SELECT DATE(bc.created_at,cn.timezone) AS fecha,
           cn.country_name AS country,
           IFNULL(u.city,'-') AS city,
           bc.campaign_title AS campaign,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount,
           COUNT(DISTINCT bc.id) AS vouchers,
           MAX(DATE(bc.rules.expires_at,cn.timezone)) AS finish
    FROM `peya-data-origins-pro.cl_barter.creates` AS bc
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON bc.rules.country.iso_code = cn.iso_code_2
    LEFT JOIN `peya-argentina.user_santiago_curat.city_usuarios` AS u ON bc.owner_id = u.user AND cn.country_name = u.country
    WHERE DATE(bc.created_at) >= DATE_ADD(CURRENT_DATE(),INTERVAL -4 DAY)
          AND UPPER(bc.campaign_title) NOT LIKE '%%OTHER_CC_%%'
          AND UPPER(bc.campaign_title) NOT LIKE '%%OTHER_AUTOCOMP_%%'
          AND UPPER(bc.campaign_title) NOT LIKE '%%OTHER_EMPLOYEE_%%'
          AND UPPER(bc.campaign_title) NOT LIKE '%%OTHER_OTHER_%%'
          AND UPPER(bc.campaign_title) NOT LIKE '%%OTHER_NCR_%%'
    GROUP BY 1,2,3,4,5,6,7,8)
SELECT DISTINCT bt.country AS Country,
       bt.city AS City,
       bt.campaign AS Campaign,
       CONCAT('Disc Amount: ',bt.discount_amount,' - Max Amount: ',bt.discount_max_amount,' - MOV: ',bt.min_amount) AS Variation,
       MAX(bt.finish) AS Finish,
       SUM(IFNULL(bt.vouchers,0)) AS Vouchers
FROM barter_table AS bt
WHERE bt.fecha >= DATE_ADD(CURRENT_DATE(),INTERVAL -3 DAY)
GROUP BY 1,2,3,bt.discount_amount,bt.discount_max_amount,bt.min_amount'''

In [11]:
# Descago la data
bq_crudo = pd.io.gbq.read_gbq(q, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 28226/28226 [00:04<00:00, 5976.79rows/s]


In [12]:
# Copio la data
crudo = bq_crudo.copy()

### TRABAJO

In [13]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','City','Variation','Finish']
cols_float = [i for i in crudo.columns if i not in cols_str]
crudo[cols_float] = crudo[cols_float].astype(float)
crudo[cols_str] = crudo[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
crudo['Campaign'] = crudo['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
crudo['Finish'] = pd.to_datetime(crudo['Finish'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')
# Hago un listado de los paises
countries = list(crudo['Country'].unique())
# Ordeno la base
crudo.sort_values(by=['Country','Campaign','City','Vouchers'],inplace=True)

In [14]:
# Creo las columnas para los segmentos
crudo['Benefit'] = crudo['Campaign'].apply(lambda x: filtros(x,'BENEFICIOS'))
crudo['Type'] = crudo['Campaign'].apply(lambda x: filtros(x,'TIPOS'))
crudo['Segment'] = crudo['Campaign'].apply(lambda x: filtros(x,'SEGMENTOS'))
crudo['Level'] = crudo['Campaign'].apply(lambda x: filtros(x,'NIVELES'))
crudo['Campaña'] = crudo['Campaign'].apply(lambda x: filtros(x,'CAMPAIGNS'))
crudo['Automated'] = crudo['Campaign'].apply(lambda x: filtros(x,'AUTOMATIONS'))
crudo['Budget'] = crudo['Campaign'].apply(lambda x: filtros(x,'BUDGET'))

In [15]:
# Hago una PT por Ciudad sacado las Variations
index = ['Country','City','Benefit','Type','Segment','Level','Campaña','Automated','Budget']
values = ['Vouchers','Finish']
pt_active = crudo.pivot_table(index=index,values=values,aggfunc={'Vouchers':np.sum,'Finish':np.max}).reset_index()

In [16]:
# Divido los DF por Country
active_arg = pt_active[pt_active['Country'] == 'ARGENTINA'].copy()
active_chi = pt_active[pt_active['Country'] == 'CHILE'].copy()

### CARGA

In [15]:
# Carga Argentina
carga(active_arg,'1NJtY4CpejWDyEDSBOFEdWJIVgq7Fvd_U0O6UlUBZuUk','Crudo Active')

Carga Correcta!


In [7]:
# Carga Chile
carga(active_chi,'1WrgzvPGpTIqK5GQiMEk4FkfUB7LVE21d1BI670yx_Os','Crudo Active')